In [1]:
import numpy as np
import xarray as xr
import rasterio
%matplotlib inline
from matplotlib.pyplot import *
from glob import glob
import os
import datetime

from rasterio_to_xarray import rasterio_to_xarray, xarray_to_rasterio, xarray_to_rasterio_by_band

In [5]:
def maiac_file_to_da(filename):
    da = rasterio_to_xarray(filename)
    
    da.values[da.values == -28672] = np.nan
    da.values[da.values == 0] = np.nan
    
    #da.values = da.values.astype(np.float64)
    
    time_str = os.path.basename(filename)[17:28]
    #print(time_str)
    time_obj = datetime.datetime.strptime(time_str, '%Y%j%H%M')
    da.coords['time'] = time_obj
    
    return da

In [6]:
files = sorted(glob('ForVFPoC/2003/Projected/*_projPM25.tif'))

In [7]:
list_of_das = map(maiac_file_to_da, files)

In [8]:
res = xr.concat(list_of_das, 'time')

In [9]:
newres = res.isel(time=np.argsort(res.time))

In [10]:
r = newres.resample('D', dim='time', how='max')

In [11]:
r = r.dropna(dim='time', how='all')

In [12]:
overall_mean = r.mean(dim='time', keep_attrs=True)

Here comes the rasterstats stuff

In [23]:
import geopandas as gpd
import pandas as pd

In [18]:
import rasterstats

In [16]:
gdf = gpd.GeoDataFrame.from_file('nuts3_OSGB.json')

In [20]:
aff = rasterio.Affine.from_gdal(*r.attrs['affine'])

In [22]:
res = rasterstats.zonal_stats(gdf, overall_mean.values, affine=aff)

/Users/robin/anaconda3/lib/python3.5/site-packages/rasterstats/io.py:292: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


In [24]:
gdf.join(pd.DataFrame(res))

,NUTS312CD,NUTS312NM,geometry,count,max,mean,min
0,UKC11,Hartlepool and Stockton-on-Tees,"(POLYGON ((445109.2966476632 519101.399278637,...",193,NaN,NaN,NaN
1,UKC12,South Teesside,(POLYGON ((454949.8977416072 526672.2009643548...,190,NaN,NaN,NaN
2,UKC13,Darlington,"POLYGON ((423496.5979902297 524724.2989774863,...",122,24.111914,20.057585,16.194729
3,UKC14,Durham CC,"POLYGON ((417139.1983994703 558194.7978395412,...",1414,NaN,NaN,NaN
4,UKC21,Northumberland,"(POLYGON ((429470.097829263 604755.7993969744,...",3182,NaN,NaN,NaN
5,UKC22,Tyneside,(POLYGON ((437549.5981616732 567883.8981630676...,254,NaN,NaN,NaN
6,UKC23,Sunderland,(POLYGON ((441408.9968317673 557945.0975113782...,89,NaN,NaN,NaN
7,UKD11,West Cumbria,(POLYGON ((323342.5985472992 463558.1993230862...,1319,NaN,NaN,NaN
8,UKD12,East Cumbria,(POLYGON ((335629.3984283255 472950.9980705814...,3005,36.134338,15.276911,5.178984
9,UKD31,Greater Manchester South,"POLYGON ((384292.696604282 404709.299198525, 3...",352,38.154350,20.867738,13.010757


In [13]:
arr = r.isel(time=[0,1,2,3,4]).values

In [16]:
rasterstats.zonal_stats('nuts3_OSGB.json', overall_mean.values, affine=aff, nodata=np.nan, raster_out=True)

[{'count': 193,
  'max': nan,
  'mean': nan,
  'min': nan,
  'mini_raster_affine': Affine(1256.543044095589, 0.0, 434557.7179944995,
       0.0, -1256.543044095589, 538388.7937454027),
  'mini_raster_array': masked_array(data =
   [[-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --]
   [-- -- -- -- -- -- -- -- -- nan nan nan -- -- -- --]
   [-- -- -- -- -- -- -- -- nan nan nan nan nan -- -- --]
   [-- -- -- -- -- -- -- -- nan nan nan nan nan nan nan --]
   [-- -- -- -- -- -- -- nan nan nan nan nan nan nan -- --]
   [-- -- -- -- -- -- -- nan nan nan nan nan nan nan -- --]
   [-- -- -- -- -- -- 19.90658187866211 16.898897171020508 nan nan nan nan
    nan nan -- --]
   [-- -- -- -- -- -- 19.84136390686035 16.91560935974121 nan nan nan nan nan
    nan nan --]
   [-- -- -- -- 19.94559669494629 19.86078643798828 20.204187393188477
    20.204376220703125 nan nan nan nan nan nan nan --]
   [-- -- -- -- 19.880210876464844 20.22394371032715 20.18977165222168
    18.094236373901367 nan nan nan nan 